# Proper Orthogonal Decomposition (POD) for Structural Mechanics
inspired from Buljak, V. (2012). Inverse Analyses with Model Reduction Proper Orthogonal Decomposition in Structural Mechanics. Springer Berlin Heidelberg (Vol. 33). https://doi.org/10.1007/978-3-642-22703-5

## Use Case : Frame made up of plane elastic trusses (11 rods et 5 unconstrained nodes with 2 degrees of freedom (10 dof)). The displacement is fixed at node 6 and node 7 fixée. The load is applied on nodes 1&2 with respectivelely a force $F1 = -5000 N$ and a force $F2 = -2000 N$ (see figure below)  

## Frame

<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/TrussStructure_LBC.png" alt="Drawing" style="width: 800px;"/>


In [ ]:
using LinearAlgebra, CSV, DataFrames, DelimitedFiles,Plots # needed for the decomposition operations

## Definition of the stiffness matrix function for a plane elastic truss element 

In [ ]:
function TrussElementStiffness(coo,E,A)
# Function that computes stiffness matrix for truss element
# (file name trstiff.m)
L=sqrt((coo[2,1]-coo[1,1])^2+(coo[2,2]-coo[1,2])^2)
C=(coo[2,1]-coo[1,1])/L
S=(coo[2,2]-coo[1,2])/L
# Stiffness matrix
STF=E*A/L*[C^2 S*C -C^2 -S*C;
S*C S^2 -S*C -S^2;
-C^2 -S*C C^2 S*C;
-S*C -S^2 S*C S^2];
return(STF)
end

## Assemble&Solve Function

In [ ]:
# Main program for truss structure
# Coordinates - units L[mm]
function TrussAssembleSolve(E,A)
COO=[0 0; 2 0; 4 0 ;1 1; 3 1; -1 1; 5 1]
COO=1000*COO
DOF=size(COO,1)*2
# Elements
ELM=[1 2; 2 3; 6 4; 4 5; 5 7; 1 6; 1 4; 2 4; 2 5; 3 5; 3 7]
# Properties (units F[N], L[mm])
#E=125000
#A=6
# Assembling of stiffness matrix
MSTF=zeros(DOF,DOF)
X=zeros(2,2)
for i in 1:size(ELM,1)
X[1,:]=COO[ELM[i,1],:]
X[2,:]=COO[ELM[i,2],:]
STF=TrussElementStiffness(X,E,A)
MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,1]-
1:2*ELM[i,1]]=MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,1]-
1:2*ELM[i,1]]+STF[1:2,1:2]
MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,2]-
1:2*ELM[i,2]]=MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,2]-
1:2*ELM[i,2]]+STF[3:4,1:2]
MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,1]-
1:2*ELM[i,1]]=MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,1]-
1:2*ELM[i,1]]+STF[1:2,3:4]
MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,2]-
1:2*ELM[i,2]]=MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,2]-
1:2*ELM[i,2]]+STF[3:4,3:4]
end
# Constrains
con=[12 13 14 15] # constrained DOF
CDOF=size(con,2)
# Forces
dF=zeros(DOF,1)
dF[2]=-5000
dF[6]=-2000
# Reduced stiffness matrix & force vector
RSTF=MSTF[1:DOF-CDOF,1:DOF-CDOF]
dFR=dF[1:DOF-CDOF]
# Solving for displacements
d=inv(RSTF)*dFR
# End of main program
return(d)
end

## Definition of the design of experiments function (doe)

In [ ]:
function doe()
    Y=zeros(10,16)
    P=zeros(2,16)
    cnt=0
    for E in [125000 150000 175000 200000]
        for A in [6 8 10 12]
            Disp = TrussAssembleSolve(E, A)
            #show(Disp)
            cnt += 1
            Y[:,cnt] = Disp
            P[:,cnt] = [E;A]
        end
    end
    #show(U)
    return (Y,P)
end

## Definition of the calculation function for the matrix $B$ of coefficients of interpolation 

In [ ]:
function BlinMtx(A, p)
    # Interpolation by the use of RBFs [Linear splines type]
    # Normalization of P [0 1]
    M = size(p, 1)
    N = size(p, 2)
    minP = zeros(M)
    maxP = zeros(M)
    x = zeros(M, N)
    G = zeros(N, N)
    for j = 1:M
        minP[j] = minimum(p[j,:])
        maxP[j] = maximum(p[j,:])
        for i = 1:N
            x[j,i] = (p[j,i] - minP[j]) / (maxP[j] - minP[j])
        end
    end
    for i = 1:N
        for j = 1:N
            G[i,j] = sum((x[:,i] - x[:,j]).^2).^0.5
        end
    end
    return A * inv(G)
end

## Definition of the calculation function for the vector $G$ of interpolation functions

In [ ]:
function GlinVec(p, pX)
    # Function that constructs G vector as function of given parameters
    M=size(p,1)
    N = size(p, 2); # The number of generated snapshots
    minP = zeros(M)
    maxP = zeros(M)
    x = zeros(M, N)
    G = zeros(N)
    # Normalization of p
    for j = 1:M
        minP[j,1] = minimum(p[j,:])
        maxP[j,1] = maximum(p[j,:])
        for i = 1:N
            x[j,i] = (p[j,i] - minP[j]) / (maxP[j] - minP[j])
        end
    end
    gi(x,y) = (sum((x-y).^2).^0.5)
    value = pX
    for k = 1:N
        G[k] = gi(value,x[:,k])
    end
    return G
end

## DOE execution for the snapshot  $Y$ and the parameter $P$ matrices calculation 
<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/Snapshotmatrix4FEM.png" alt="Drawing" style="width: 700px;"/>

$Y=\left[\begin{array}{cccccccccc}
| & | & | & | & | & | & | & | & | & |\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
| & | & | & | & | & | & | & | & | & |\\
y_{1} & y_{2} & y_{3} & y_{4} & y_{5} & y_{6} & y_{7} & y_{8} & y_{9} & y_{10}\\
| & | & | & | & | & | & | & | & | & |\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
| & | & | & | & | & | & | & | & | & |
\end{array}\right]$

Range of Young modulus $E = \{125000; 150000; 175000; 200000\} (unit\ MPa)$
Range of cross section $A = \{6; 8; 10; 12\} (unit\ mm)$

In [ ]:
# Execute The design of experiments (DOE)
Y,P=doe()
# Wriing data into csv files
CSV.write("data/X.csv",DataFrame(P'))
CSV.write("data/Y.csv",DataFrame(Y))
println("\rThe Data - Snapshot matrix coming out from the DOE Y =\r")
show(stdout,"text/plain", round.(Y;digits=2))
println("\n\nThe Parameter matrix P coming out from the DOE P =\r")
show(stdout,"text/plain", round.(P;digits=0))


## Verification of the results for the first column ($E = 125000 MPa$ and $A = 6mm$) with the commercial finite element software MSC.Marc
<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/TrussStructure_DisplacementX.png" alt="Drawing" style="width: 800px;"/>
<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/TrussStructure_DisplacementY.png" alt="Drawing" style="width: 800px;"/>

## POD 2 methodes are used here which give the same results (1. eigen decomposition and 2. singular value decompostion) followed by reconstruction and reduction steps

1. Eigen decomposition

In [ ]:
F=eigen(Y*Y')
println("\rEigenValues vector λ =\r")
round.(F.values,digits=2)

In [ ]:
println("\rEigenVectors matrix ϕ =\r")
ϕ=F.vectors
round.(ϕ;digits=2)

In [ ]:
println("\rAmplitude matrix A =\r")
A=F.vectors'*Y
round.(A;digits=2)

In [ ]:
println("\rReconstruction of Yrecons = ϕA\r")
Yrecons=ϕ*A
round.(Yrecons;digits=2)

In [ ]:
println("\rReduction of Yapprox = ϕ10*A10'≈ Y\r")
Yapprox=ϕ[:,10]*A[10,:]'
round.(Yapprox;digits=2)

In [ ]:
y1=ϕ[:,10]*A[10,:][1]
println("\ry1 = \r")
round.(y1;digits=2)

In [ ]:
y2=ϕ[:,10]*A[10,:][2]
println("\ry2 = \r")
round.(y2;digits=2)

2. SVD

In [ ]:
#Calculate the SVD of the snapshot matrix
Z=svd(Y)
println("\rSVD decomposition of the snaphot matrix=\r")
Z

* Modes

In [ ]:
println("\rSVD modes \r")
round.(Z.U;digits=2)

* Mode coefficients

In [ ]:
println("\rSVD modes coefficients\r")
round.(diagm(Z.S)*Z.Vt;digits=2)

In [ ]:
println("\rReconstruction of Yrecons = UΣV* = \r")
Yrecons=Z.U*diagm(Z.S)*Z.Vt
round.(Yrecons;digits=2)

In [ ]:
println("\rThe approximation Ỹ is based on the first most energetic mode")
println("\rỸ = ")
Ỹ=Z.U[:,1]*(diagm(Z.S)*Z.Vt)[1,:]'
round.(Ỹ;digits=2)

In [ ]:
y1=Z.U[:,1]*(Z.S[1]*Z.Vt[1,:][1])
println("\r y1 = \r")
round.(y1;digits=2)

In [ ]:
y2=Z.U[:,1]*(Z.S[1]*Z.Vt[1,:][2])
println("\ry2 = \r")
round.(y2;digits=2)

# Interpolation
For arbitray parameters, the combined use of radial basis functions (RBF) is required

As a reminder
* Parameter matrix $P$

In [ ]:
println("\rP=\r")
show(stdout,"text/plain", round.(P;digits=0))

* Amplitude matrix $A$

In [ ]:
println("\rA = \r")
round.(A;digits=2)

* POD mode matrix $\phi$

In [ ]:
println("\rϕ = \r")
round.(ϕ;digits=2)

* Interpolation coefficient matrix $B$

In [ ]:
B=BlinMtx(A,P)
round.(B;digits=2)

* Calculation of the nodal displacement vector for the normalized point $Pint$ to be interpolated ($E = 135000 Mpa ; A = 11 mm^{2}$)

In [ ]:
p=scatter(zeros(1),zeros(1),c=:blue, leg=false,xformatter=x->string(Int(x/1e3)), xlims=(120000,200500),ylims=(5,13),bg=:darkorange);
for i in 1:16
scatter!([P[:,i][1]],[P[:,i][2]],c=:blue, leg=false);
end
scatter!([135000.],[11.], shape=:square, title="Design of Experiments", xlabel="Young modulus E",ylabel="Cross section A");
savefig("img/doe.png");
display(p)

In [ ]:
Pint=[(135-125)/(200-125);(11-6)/(12-6)]
round.(Pint;digits=2);

* Linear spline interpolation functions vector $g$

In [ ]:
g=GlinVec(P,Pint)
round.(g;digits=2)

Results of the interpolated nodal displacement vector

In [ ]:
ũ=ϕ*B*g
round.(ũ;digits=2)

to be compared to the result of the full high fidelity fem results

In [ ]:
ufem=TrussAssembleSolve(135000,11)
round.(ufem;digits=2)

In [ ]:
writedlm("data/uinter.txt",ũ,',')
writedlm("data/ufem.txt",ufem,',')

In [ ]:
p=plot(Y, xlims=(1,10),legend=false,show=true)
p=plot!(ũ,shape =:square,xlims=(1,10),legend=false)
scatter!(ufem,bg=:darkorange)
display(p)
savefig("img/interpolation.png")